In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
import re
class Valve:
    def __init__(self, name, flow, connections):
        self.name = name
        self.flow = int(flow)
        self.connections = connections
        if int(flow) == 0:
            self.valve_on = True
        else:
            self.valve_on = False
        
    def __str__(self):
        return f'{self.name} flow {self.flow} and goes to {",".join(self.connections)}'

In [55]:
from copy import deepcopy

class ValveState:
    def __init__(self, location, valves, time_left=30):
        self.cur_loc = location
        self.time_left = time_left
        self.valves = valves
        self.cost_map =  self._create_paths(valves)
        self.total_flow=0
        self.cur_flow=0
        
        self.releases = []
        self.nvalves_open = sum([s.valve_on for s  in valves.values()])
        self.valves_closed = [s.name for s  in valves.values() if not s.valve_on]
        
        
    def _get_minutes_transition(self, start,end, valves):
        search = deque([start])
        steps_so_far = {start : 0}
        
        while len(search) > 0 :
            current = search.popleft()
            
            if current == end:
                return steps_so_far[end]
            
            for next_state in valves[current].connections:
                nsteps = steps_so_far[current]+1
                
                if next_state not in steps_so_far.keys() or nsteps < steps_so_far[next_state]:
                    steps_so_far[next_state] = nsteps
                    search.append(next_state)
                    
    def _create_paths(self,valves):
        cost_to_traverse = {}
        for s in valves.keys():
            cost_to_traverse[s] = {}
            for e in valves.keys():
                if s==e:
                    continue
                cost_to_traverse[s][e] = self._get_minutes_transition(s,e,valves)
        return cost_to_traverse
    
    def is_state_complete(self):
        remaining = self.valves_closed
        if len(remaining) <1:
            return True
        icur =  self.cur_loc
        itime = self.time_left
        
        if (itime < min([self.cost_map[icur][e]+1 for e in remaining])):
            return True
        return False
    
    
    def next_possible_states(self):
        neigh = deepcopy(self)
        remaining = self.valves_closed
        if len(remaining)<1:
            return []
        
        itime = self.time_left
        icur =  self.cur_loc
       
        neighbor_states =[]
        if itime < min([self.cost_map[icur][e]+1 for e in remaining]):
            return []
                
        for imove in remaining:
            if itime < self.cost_map[icur][imove]+1 :
                continue
            n=deepcopy(neigh)
            n.valves[imove].valve_on = True
            n.nvalves_open +=1  
            n.valves_closed.remove(imove)
            i_new_time = itime - self.cost_map[icur][imove] -1
            n.total_flow  += n.valves[imove].flow * (i_new_time)
            n.releases.append((n.valves[imove].flow , (i_new_time)))
            n.cur_loc = imove
            n.time_left = i_new_time
            neighbor_states.append(n)
        return neighbor_states
        
    def __lt__(self, other):
        return (self.time_left,-1*self.total_flow) < (other.time_left,-1*other.total_flow)
        

In [13]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, ValveState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: ValveState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> ValveState:
        return heapq.heappop(self.elements)[1]

In [140]:
from collections import deque

def create_paths_one(initial_state):
    frontier = PriorityQueue()
    
    seen =set([])
    ends=[]
    total_valvs = len(initial_state.valves)
    frontier.put(initial_state,0)
    
    while not frontier.empty():
        current = frontier.get()
        
        if current.is_state_complete() or current.nvalves_open == total_valvs :
            ends.append(current)
        
        for next_state in current.next_possible_states():
            
            tag = (next_state.total_flow,next_state.nvalves_open, )
            if tag not in seen:
                seen.add(tag)
                priority = 1 # (sum([next_state.valves[v].flow for v in next_state.valves_closed]))
                frontier.put(next_state, priority)
    
    max_rel = max([ e.total_flow for e in ends])
    for e in ends:
        if max_rel == e.total_flow :
            return max_rel, e
#    return [ e.total_flow for e in ends]

In [141]:
def read_file(filename):
    str_format= 'Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)'
    valves={}
    with open(filename, 'r') as f :
        for line in f.readlines():
            m = re.match(str_format,line)
            vals = m.groups()
            neighs = [x.strip() for x in vals[2].split(',')]
            valves[vals[0]] = Valve(vals[0], vals[1], neighs)
    return valves

In [142]:
def part_one(filename):
    initial_valves = read_file(filename)
    initial_state = ValveState( 'AA', initial_valves)
    max_rel, e = create_paths_one(initial_state )
    return max_rel

In [143]:
%%time 
part_one('input/test_16.txt')

CPU times: user 319 ms, sys: 3.69 ms, total: 323 ms
Wall time: 323 ms


1651

In [144]:
%%time 
part_one('input/day_16.txt')

CPU times: user 25.5 s, sys: 95 ms, total: 25.6 s
Wall time: 25.6 s


1751

# Part 2

In [12]:
from copy import deepcopy
from collections import deque

class ValveStateElephant:
    def __init__(self, location, valves, time_left=26):
        self.cur_loc = [location,location]
        self.time_left = [time_left, time_left]
        
        
        self.valves = valves
        
        self.cost_map =  self._create_paths(valves)
        self.total_flow=0
        self.cur_flow=0
        
        self.releases = []
        self.nvalves_open = sum([s.valve_on for s  in valves.values()])
        self.valves_closed = [s.name for s  in valves.values() if not s.valve_on]
        
        
    def _get_minutes_transition(self, start,end, valves):
        search = deque([start])
        steps_so_far = {start : 0}
        
        while len(search) > 0 :
            current = search.popleft()
            
            if current == end:
                return steps_so_far[end]
            
            for next_state in valves[current].connections:
                nsteps = steps_so_far[current]+1
                
                if next_state not in steps_so_far.keys() or nsteps < steps_so_far[next_state]:
                    steps_so_far[next_state] = nsteps
                    search.append(next_state)
                    
    def _create_paths(self,valves):
        cost_to_traverse = {}
        for s in valves.keys():
            cost_to_traverse[s] = {}
            for e in valves.keys():
                if s==e:
                    continue
                cost_to_traverse[s][e] = self._get_minutes_transition(s,e,valves)
        return cost_to_traverse
                
        
    def set_state(self,iloc, jloc):
        self.cur_loc[0] = iloc
        self.cur_loc[1] = jloc
    
    def is_state_complete(self):
        remaining = self.valves_closed
        if len(remaining) <1:
            return True
        icur =  self.cur_loc[0]
        jcur =  self.cur_loc[1]
        itime = self.time_left[0]
        jtime = self.time_left[1]
        
        if ((itime < min([self.cost_map[icur][e]+1 for e in remaining]))
            and (jtime < min([self.cost_map[jcur][e]+1 for e in remaining]) )):
            return True
        return False
        
        
    def next_possible_states(self):
        neigh = deepcopy(self)
        remaining = self.valves_closed
        if len(remaining)<1:
            return []
        
        itime = self.time_left[0]
        jtime = self.time_left[1]
        
        icur =  self.cur_loc[0]
        jcur =  self.cur_loc[1]
       
        neighbor_states =[]
        if itime < min([self.cost_map[icur][e]+1 for e in remaining]):
            imove=icur
            for jmove in remaining:
                if jtime < self.cost_map[jcur][jmove]+1 :
                    continue
                n = deepcopy(neigh)
                n.valves[jmove].valve_on = True
                n.nvalves_open +=1  
                n.valves_closed.remove(jmove)
                new_time = jtime- self.cost_map[jcur][jmove] -1
                n.total_flow  += n.valves[jmove].flow * (new_time)
                n.releases.append((n.valves[jmove].flow , (new_time)))
                n.set_state(imove, jmove)
                n.time_left = [itime, new_time ] 
                neighbor_states.append(n)
                
        for imove in remaining:
            if itime < self.cost_map[icur][imove]+1 :
                continue
            n=deepcopy(neigh)
            n.valves[imove].valve_on = True
            n.nvalves_open +=1  
            n.valves_closed.remove(imove)
            i_new_time = itime- self.cost_map[icur][imove] -1
            n.total_flow  += n.valves[imove].flow * (i_new_time)
            n.releases.append((n.valves[imove].flow , (i_new_time)))
            
            if jtime < min([self.cost_map[jcur][e]+1 for e in remaining]):
                n.set_state(imove, jcur)
                n.time_left = [i_new_time, jtime ] 
                neighbor_states.append(n)
                continue
                
            for jmove in remaining:
                if jmove == imove:
                    continue
                    
                if jtime < self.cost_map[jcur][jmove]+1 :
                    continue
            
                nn = deepcopy(n)
                nn.valves[jmove].valve_on = True
                nn.nvalves_open +=1  
                nn.valves_closed.remove(jmove)
                j_new_time = jtime- self.cost_map[jcur][jmove] -1
                nn.total_flow  += nn.valves[jmove].flow * (j_new_time)
                nn.releases.append((nn.valves[jmove].flow , (j_new_time)))
                nn.set_state(imove, jmove)
                nn.time_left = [i_new_time, j_new_time ] 
                neighbor_states.append(nn)
                
        return neighbor_states
                    
                    
        
    def __lt__(self, other):
        return self.total_flow > other.total_flow
        

In [13]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, ValveStateElephant]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: ValveStateElephant, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> ValveStateElephant:
        return heapq.heappop(self.elements)[1]

In [182]:
from collections import deque

def create_paths_two(initial_state):
#    frontier = deque([]) 
    
    frontier = PriorityQueue()
    
    seen =set([])
    ends=[]
    total_valvs = len(initial_state.valves)
#    frontier.append(initial_state)
    frontier.put(initial_state,0)
    
#    while len(frontier)>0:
    while not frontier.empty():
#        current = frontier.popleft()
        current = frontier.get()
        
        if current.is_state_complete() or current.nvalves_open == total_valvs :
            ends.append(current)
            break
        
        for next_state in current.next_possible_states():
            
            tag = (next_state.total_flow,next_state.nvalves_open)
            if tag not in seen:
                seen.add(tag)
                
                left =sum([s.flow for s in  next_state.valves.values() if not s.valve_on])
                
                priority = -1*(next_state.total_flow + left*(sum(next_state.time_left)/4))
#                priority = -1*(next_state.total_flow+ left*(sum(next_state.time_left)))
                
                frontier.put(next_state, priority)
#                frontier.append(next_state)
    
    max_rel = max([ e.total_flow for e in ends])
    for e in ends:
        if max_rel == e.total_flow :
            return max_rel, e
#    return [ e.total_flow for e in ends]

In [183]:
def part_two(filename):
    initial_valves = read_file(filename)
    initial_state = ValveStateElephant( 'AA', initial_valves)
    max_rel = create_paths_two(initial_state )
    return max_rel

In [184]:
%%time
t,e = part_two('input/test_16.txt')
t
#1707

CPU times: user 26.6 ms, sys: 1.4 ms, total: 28 ms
Wall time: 27.8 ms


1707

In [185]:
%%time
t,e = part_two('input/day_16.txt')
t

CPU times: user 58.2 s, sys: 323 ms, total: 58.6 s
Wall time: 58.6 s


2207

In [ ]:
#1837 is wrong :/ 
#2154 is too low as well -_- 